In [ ]:
#! git clone https://github.com/Tikquuss/sag_torch

In [ ]:
#%cd sag_torch

In [1]:
#! pip install -r requirements.txt

In [2]:
import os

from src.modeling import Model

In [3]:
%load_ext tensorboard

# Train

In [ ]:
#! wandb login $som_key

## cmd

##### On run

In [ ]:
! chmod +x train.sh 
! ./train.sh 

In [ ]:
%tensorboard --logdir /content/log_files/0/classification_tdf=80-wd=0.0-r_lr=0.001-d_lr=0.001-r_d=0.0-d_d=0.0-opt=adam/lightning_logs

In [ ]:
pretrained_filename = "/content/log_files/0/classification_tdf=80-wd=0.0-r_lr=0.001-d_lr=0.001-r_d=0.0-d_d=0.0-opt=adam/epoch=1-val_loss=5.2339.ckpt"
model = Model.load_from_checkpoint(pretrained_filename)

In [ ]:
#! rm -r /content/log_files/0

##### Multiple run (for phase diagram)

In [ ]:
! chmod +x train_loop.sh
! ./train_loop.sh 

## Without cmd (see multiple_runs.py) 

In [ ]:
from src.utils import AttrDict
from src.dataset import get_dataloader
from src.trainer import train

In [ ]:
train_pct=80
weight_decay=0.0
representation_lr=0.001
decoder_lr=0.001
representation_dropout=0.0
decoder_dropout=0.0
opt="adam"

group_name=f"tdf={train_pct}-wd={weight_decay}-r_lr={representation_lr}-d_lr={decoder_lr}-r_d={representation_dropout}-d_d={decoder_dropout}-opt={opt}"

random_seed=0
operator="+"
modular=False

log_dir="../log_files"

p = 100
task = "classification"

params = AttrDict({
    ### Main parameters
    "task" : task,
    "exp_id" : f"{task}_{group_name}",
    "log_dir" : f"{log_dir}/{random_seed}",

    ### Model
    "emb_dim" : 256, 
    "hidden_dim" : 512,  
    "n_layers" : 1,
	"representation_dropout" : representation_dropout,
	"decoder_dropout" : decoder_dropout,
    "pad_index" : None, 
    "p" : p, 

    ### Dataset
    "operator" : operator, 
    "modular" : modular,
    "train_pct" : train_pct,
    "batch_size" : 512,

    ### Optimizer
    "optimizer" : f"{opt},weight_decay={weight_decay},beta1=0.9,beta2=0.99,eps=0.00000001",
    "representation_lr" : representation_lr,
    "decoder_lr" : decoder_lr,

    ### LR Scheduler
    "lr_scheduler" : None,
    #"lr_scheduler" : "reduce_lr_on_plateau,factor=0.2,patience=20,min_lr=0.00005,mode=min,monitor=val_loss",
    
    ### Training
    "max_epochs" : 2, 
    "validation_metrics" : "val_loss",
    "checkpoint_path" : None, 
    "model_name": "", 
    "every_n_epochs":1, 
    "every_n_epochs_show":1, 
    "early_stopping_patience":1e9, 
    "save_top_k":-1,

    # Wandb 
    "use_wandb" : False,
	"wandb_entity" : "grokking_ppsp",
	"wandb_project" : f"toy_model_grokking_op={operator}-p={p}-task={task}-mod={modular}",
    "group_name" : group_name,

    "group_vars" : None,

    ### Intrinsic Dimension Estimation
    #"ID_params" : {},
    #"ID_params": {"method" : "mle", "k":2},
    "ID_params": {"method" : "twonn"},
    
    # Devices & Seed
    "accelerator" : "auto",
    "devices" : "auto",
    "random_seed": random_seed,

    ### Early_stopping (for grokking) : Stop the training `patience` epochs after the `metric` has reached the value `metric_threshold` 
    #"early_stopping_grokking" : None,
    "early_stopping_grokking" : "patience=int(1000),metric=str(val_acc),metric_threshold=float(90.0)",

})
params["weight_decay"] = weight_decay
params["regression"] = task == "regression"
train_loader, val_loader, dataloader, data_infos = get_dataloader(
    p, train_pct, regression = params.regression, operator=params.operator, 
    modular = params.modular, batch_size=params.batch_size, num_workers=2
)
print(data_infos)
params["data_infos"] = data_infos

##### On run

In [ ]:
model, result = train(params, train_loader, val_loader)

In [ ]:
%tensorboard --logdir /content/log_files/0/classification_tdf=80-wd=0.0-r_lr=0.001-d_lr=0.001-r_d=0.0-d_d=0.0-opt=adam/lightning_logs

In [ ]:
#! rm -r /content/log_files/0